In [ ]:
!pip install python-telegram-bot --upgrade
!pip install dextools-python

In [1]:
api_key = "b5ZO15hnCV9dnJr7XOQZIaGMWXCb6y5J7bXSoJvS"

In [2]:
from dextools_python import DextoolsAPIV2
import pandas as pd
import time
import datetime

dextools = DextoolsAPIV2(api_key, plan="trial")

In [3]:
# Get Tokens

In [4]:
def get_liquidity_pools_sorted_by_date_df(chain='solana', start_date=None, end_date=None, num_pages=3):
    if not start_date:
        start_date = datetime.datetime.now()
    if not end_date:
        end_date = start_date - datetime.timedelta(hours=6)
    
    start_date_str = start_date.strftime('%Y-%m-%dT%H:%M:%S')
    end_date_str = end_date.strftime('%Y-%m-%dT%H:%M:%S')
    
    all_tokens = []
    for page in range(0, num_pages):
        response = dextools.get_pools(chain, from_=end_date_str, to=start_date_str, page=page, order="desc")
        print(response)
        all_tokens.extend(response["data"]['results'])
        time.sleep(1.5)
    
    # Flatten the nested 'socialInfo' and convert data to DataFrame
    tokens_df = pd.json_normalize(all_tokens)
    
    return tokens_df

In [5]:
def run_get_tokens():
    tokens = get_liquidity_pools_sorted_by_date_df()
    return tokens

In [6]:
tokens = run_get_tokens()
"Complete"

{'statusCode': 200, 'data': {'page': 0, 'pageSize': 20, 'totalPages': 25, 'results': [{'creationBlock': 259266148, 'creationTime': '2024-04-09T23:27:18.789Z', 'exchange': {'name': 'Raydium', 'factory': '675kpx9mhtjs2zt1qfr1nyhuzelxfqm9h24wfsut1mp8'}, 'address': 'G9tSict35C1gCRa4pjQkt7kCjYUTCzCKPDxP12edQ71x', 'mainToken': {'name': 'Dr Coin', 'symbol': 'DRCOIN', 'address': '3qXtXwVdZpxJmD9LmDHoZcKQHcJED3mnKYaGYo5hPCYj'}, 'sideToken': {'name': 'Wrapped SOL', 'symbol': 'SOL', 'address': 'So11111111111111111111111111111111111111112'}}, {'creationBlock': 259266082, 'creationTime': '2024-04-09T23:26:52.482Z', 'exchange': {'name': 'Raydium', 'factory': '675kpx9mhtjs2zt1qfr1nyhuzelxfqm9h24wfsut1mp8'}, 'address': '3DCEoAw85qEaaxYy99gFeuiU8AatY2Ff2tJ72UkAJAJZ', 'mainToken': {'name': 'PLONK', 'symbol': 'PLONK', 'address': '4SyB9UwYrBBVtgipMz5g8PP8gWKmg1hgnKuLoz59PzsX'}, 'sideToken': {'name': 'Wrapped SOL', 'symbol': 'SOL', 'address': 'So11111111111111111111111111111111111111112'}}, {'creationBlock

'Complete'

In [7]:
display(tokens)

,creationBlock,creationTime,address,exchange.name,exchange.factory,mainToken.name,mainToken.symbol,mainToken.address,sideToken.name,sideToken.symbol,sideToken.address
0,259266148,2024-04-09T23:27:18.789Z,G9tSict35C1gCRa4pjQkt7kCjYUTCzCKPDxP12edQ71x,Raydium,675kpx9mhtjs2zt1qfr1nyhuzelxfqm9h24wfsut1mp8,Dr Coin,DRCOIN,3qXtXwVdZpxJmD9LmDHoZcKQHcJED3mnKYaGYo5hPCYj,Wrapped SOL,SOL,So11111111111111111111111111111111111111112
1,259266082,2024-04-09T23:26:52.482Z,3DCEoAw85qEaaxYy99gFeuiU8AatY2Ff2tJ72UkAJAJZ,Raydium,675kpx9mhtjs2zt1qfr1nyhuzelxfqm9h24wfsut1mp8,PLONK,PLONK,4SyB9UwYrBBVtgipMz5g8PP8gWKmg1hgnKuLoz59PzsX,Wrapped SOL,SOL,So11111111111111111111111111111111111111112
2,259266025,2024-04-09T23:26:16.130Z,BmsttAGYsmbnER1cuALxBhy72UmnxZFA7sot3dfckaPC,Raydium,675kpx9mhtjs2zt1qfr1nyhuzelxfqm9h24wfsut1mp8,AWyvernW sinnings,WSSW,6yjStMT5S9wcqkk1HXuPK8SnMWUK46dD95fUZBveJnzv,Wrapped SOL,SOL,So11111111111111111111111111111111111111112
3,259266011,2024-04-09T23:26:09.760Z,68t8tR9qv7VNFsQRREnCxHfG97VUZhm2TxQxpcVVda2t,Raydium,675kpx9mhtjs2zt1qfr1nyhuzelxfqm9h24wfsut1mp8,Sero Coin,SERO,5J2e6UDjGDdBH4FdHhXXKXsuXYgW7RqBuF2fqBZmaHk2,Wrapped SOL,SOL,So11111111111111111111111111111111111111112
4,259265995,2024-04-09T23:26:06.090Z,Fv9mRyFG3WJU46Cozcdagm5GvYXUPkuc48vuokp3cpLv,Raydium,675kpx9mhtjs2zt1qfr1nyhuzelxfqm9h24wfsut1mp8,Oaby Michi,OabyMichi,dcVDMFcYjU4cG4HP5EdKhFK4EdauELhRwvio3UYKMPd,Wrapped SOL,SOL,So11111111111111111111111111111111111111112
5,259265966,2024-04-09T23:26:00.508Z,8V2JB4ZsjPQiaonY4H2MbRyb5yoYYrvow6tdUrb49iHY,Raydium,675kpx9mhtjs2zt1qfr1nyhuzelxfqm9h24wfsut1mp8,KUROSHI,KURO,7eikqfGKZomcgdoWD43mFy2vuaqVcEnhyLTZBKNVY5fY,Wrapped SOL,SOL,So11111111111111111111111111111111111111112
6,259265964,2024-04-09T23:25:54.989Z,239BNub9veDRdPPGmGQjnrY6sMkXN6J8eYuFRrXyr6Li,Raydium,675kpx9mhtjs2zt1qfr1nyhuzelxfqm9h24wfsut1mp8,kitler,kitler,J5Un3y3LU2XBP6iycVr1Td9yn5DPWPNVjqAnSFgyda5x,Wrapped SOL,SOL,So11111111111111111111111111111111111111112
7,259265949,2024-04-09T23:25:43.822Z,3DjfzSQgsbqmxHWxhwEVgEoCABxfCRNvgwAiMHWp1z39,Raydium,675kpx9mhtjs2zt1qfr1nyhuzelxfqm9h24wfsut1mp8,Xeno PEPE,XENOPEPE,A61xFtTM4k6b6JqUEcRX6mwk8JDtWShpa17LKkb2ibmp,Wrapped SOL,SOL,So11111111111111111111111111111111111111112
8,259265936,2024-04-09T23:25:36.204Z,ERzjuJE6vWRCy99rcVYWL2KWbLDctq5zduEhmgx1fMk7,Raydium,675kpx9mhtjs2zt1qfr1nyhuzelxfqm9h24wfsut1mp8,life on earth,EARTH,8G8Ux2oMBkCCC4K68udBLSvQAcNip3JKpdUsykGDAW4Y,Wrapped SOL,SOL,So11111111111111111111111111111111111111112
9,259265929,2024-04-09T23:25:32.582Z,E5RDBdESD7G3DbTqQvGmHMaXQRW3TWkumvsATEhLEBWa,Raydium,675kpx9mhtjs2zt1qfr1nyhuzelxfqm9h24wfsut1mp8,ERagnarok Eupee,RSSSR,CZp4sPi7eTJvKQvXj9GhXwvG9a8wxWANjBiNe3Dd7rrB,Wrapped SOL,SOL,So11111111111111111111111111111111111111112


In [8]:
import requests

def get_rug_check_data(token_address):
    # Base API URL
    base_url = 'https://api.rugcheck.xyz/v1/tokens/'
    
    # Complete API URL with the token address
    api_url = f'{base_url}{token_address}/report'
    
    # Send GET request
    response = requests.get(api_url)
    
    # Check if the request was successful
    if response.status_code == 200:
        # Parse the JSON response into a pandas DataFrame
        data = response.json()
        df_top_holders = pd.json_normalize(data)
        return df_top_holders
    else:
        return f"Failed to fetch data: status code {response.status_code}"

In [9]:
# Test Get Rug Check Data
token_address = '3K3F9G7zrPd9hPnCDKbBGXwHM85CD1ewLyzNsH51Ry43'
df = get_rug_check_data(token_address)

In [10]:
pd.set_option('display.max_columns', None)
display(df)

,mint,tokenProgram,token_extensions,topHolders,freezeAuthority,mintAuthority,risks,score,lockerOwners,lpLockers,markets,rugged,tokenType,token.mintAuthority,token.supply,token.decimals,token.isInitialized,token.freezeAuthority,tokenMeta.name,tokenMeta.symbol,tokenMeta.uri,tokenMeta.mutable,tokenMeta.updateAuthority,fileMeta.description,fileMeta.name,fileMeta.symbol,fileMeta.image,transferFee.pct,transferFee.maxAmount,transferFee.authority
0,3K3F9G7zrPd9hPnCDKbBGXwHM85CD1ewLyzNsH51Ry43,TokenkegQfeZyiNwAJbNbGKPFXCWuBvf9Ss623VQ5DA,None,[{'address': 'DyWNa439guZzgca6GFwhHgFjr8iGm9Vq...,None,None,"[{'name': 'Low amount of LP Providers', 'value...",100,None,None,[{'pubkey': '7XQh6FNHiumtgjCXPiDiSnDWoEGd9ZYHr...,False,,None,999999928222742806,9,True,None,Baby Slerf,BABYSLERF,https://ipfs.coinsult.app/ipfs/QmPU63r9RMDM35w...,False,EKJCvZtJh9YMc4TycxfatAUAC4XT6sXcw91wrnLWGegN,,Baby Slerf,BABYSLERF,https://ipfs.coinsult.app/ipfs/QmeZaCHdmPSkqyj...,0,0,11111111111111111111111111111111


In [11]:
def add_top_10_holders_percentage(df):
    # Assuming df['topHolders'] is a string representation of the topHolders list
    # Convert the string to a list of dictionaries if needed
    if isinstance(df['topHolders'].iloc[0], str):
        df['topHolders'] = df['topHolders'].apply(eval)

    # Normalize the nested JSON data for topHolders and calculate the sum of the 'pct' of the top 10 holders
    top_holders_df = pd.json_normalize(df['topHolders'].iloc[0])
    top_10_holders_pct = top_holders_df.nlargest(10, 'pct')['pct'].sum()

    # Add the top 10 holders percentage to the DataFrame
    df['top_10_holders_percentage'] = top_10_holders_pct

    # Drop the original topHolders column if you want to remove it
    #df.drop(columns=['topHolders'], inplace=True)

    return df

In [12]:
df = add_top_10_holders_percentage(df)

In [13]:
df

,mint,tokenProgram,token_extensions,topHolders,freezeAuthority,mintAuthority,risks,score,lockerOwners,lpLockers,markets,rugged,tokenType,token.mintAuthority,token.supply,token.decimals,token.isInitialized,token.freezeAuthority,tokenMeta.name,tokenMeta.symbol,tokenMeta.uri,tokenMeta.mutable,tokenMeta.updateAuthority,fileMeta.description,fileMeta.name,fileMeta.symbol,fileMeta.image,transferFee.pct,transferFee.maxAmount,transferFee.authority,top_10_holders_percentage
0,3K3F9G7zrPd9hPnCDKbBGXwHM85CD1ewLyzNsH51Ry43,TokenkegQfeZyiNwAJbNbGKPFXCWuBvf9Ss623VQ5DA,None,[{'address': 'DyWNa439guZzgca6GFwhHgFjr8iGm9Vq...,None,None,"[{'name': 'Low amount of LP Providers', 'value...",100,None,None,[{'pubkey': '7XQh6FNHiumtgjCXPiDiSnDWoEGd9ZYHr...,False,,None,999999928222742806,9,True,None,Baby Slerf,BABYSLERF,https://ipfs.coinsult.app/ipfs/QmPU63r9RMDM35w...,False,EKJCvZtJh9YMc4TycxfatAUAC4XT6sXcw91wrnLWGegN,,Baby Slerf,BABYSLERF,https://ipfs.coinsult.app/ipfs/QmeZaCHdmPSkqyj...,0,0,11111111111111111111111111111111,60.894393


In [14]:
import pandas as pd
import ast
import json

def calculate_market_values(df):
    # Convert string representation of lists back into actual lists for the 'topHolders' column
    df['topHolders'] = df['topHolders'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
    
    # Initialize total percentage and liquidity
    total_pct = 0
    liquidity_usd = 0

    # Iterate over each row for topHolders to sum percentages and calculate liquidity
    for index, row in df.iterrows():
        # Calculate total percentage from topHolders
        top_holders = row['topHolders']
        total_pct += sum(holder['pct'] for holder in top_holders)
        
        # Calculate liquidity from markets (assuming 'markets' is a stringified JSON in the DataFrame)
        markets_data = json.loads(row['markets']) if isinstance(row['markets'], str) else row['markets']
        for market in markets_data:
            liquidity_usd += market['lp']['baseUSD'] + market['lp']['quoteUSD']

    
    
    # Add the calculated values as new columns to the DataFrame
    df['total_percentage_of_top_holders'] = total_pct
    df['total_liquidity_usd'] = liquidity_usd
    return df

In [15]:
df = calculate_market_values(df)

display(df)

,mint,tokenProgram,token_extensions,topHolders,freezeAuthority,mintAuthority,risks,score,lockerOwners,lpLockers,markets,rugged,tokenType,token.mintAuthority,token.supply,token.decimals,token.isInitialized,token.freezeAuthority,tokenMeta.name,tokenMeta.symbol,tokenMeta.uri,tokenMeta.mutable,tokenMeta.updateAuthority,fileMeta.description,fileMeta.name,fileMeta.symbol,fileMeta.image,transferFee.pct,transferFee.maxAmount,transferFee.authority,top_10_holders_percentage,total_percentage_of_top_holders,total_liquidity_usd
0,3K3F9G7zrPd9hPnCDKbBGXwHM85CD1ewLyzNsH51Ry43,TokenkegQfeZyiNwAJbNbGKPFXCWuBvf9Ss623VQ5DA,None,[{'address': 'DyWNa439guZzgca6GFwhHgFjr8iGm9Vq...,None,None,"[{'name': 'Low amount of LP Providers', 'value...",100,None,None,[{'pubkey': '7XQh6FNHiumtgjCXPiDiSnDWoEGd9ZYHr...,False,,None,999999928222742806,9,True,None,Baby Slerf,BABYSLERF,https://ipfs.coinsult.app/ipfs/QmPU63r9RMDM35w...,False,EKJCvZtJh9YMc4TycxfatAUAC4XT6sXcw91wrnLWGegN,,Baby Slerf,BABYSLERF,https://ipfs.coinsult.app/ipfs/QmeZaCHdmPSkqyj...,0,0,11111111111111111111111111111111,60.894393,67.097792,987183.650027


In [18]:
def add_top_holders_percentage(df):
    # Assuming the 'topHolders' column is a list of dictionaries with the 'pct' key
    for index, row in df.iterrows():
        # Sort the top holders by percentage in descending order
        sorted_holders = sorted(row['topHolders'], key=lambda x: x['pct'], reverse=True)
        # Get the top ten holders
        top_ten_holders = sorted_holders[:10]
        for i, holder in enumerate(top_ten_holders):
            # Create a new column for each top holder's percentage
            df.at[index, f'top_holder_{i+1}_pct'] = holder['pct']

    return df

In [19]:
df = add_top_holders_percentage(df)
display(df)

,mint,tokenProgram,token_extensions,topHolders,freezeAuthority,mintAuthority,risks,score,lockerOwners,lpLockers,markets,rugged,tokenType,token.mintAuthority,token.supply,token.decimals,token.isInitialized,token.freezeAuthority,tokenMeta.name,tokenMeta.symbol,tokenMeta.uri,tokenMeta.mutable,tokenMeta.updateAuthority,fileMeta.description,fileMeta.name,fileMeta.symbol,fileMeta.image,transferFee.pct,transferFee.maxAmount,transferFee.authority,top_10_holders_percentage,total_percentage_of_top_holders,total_liquidity_usd,top_holder_1_pct,top_holder_2_pct,top_holder_3_pct,top_holder_4_pct,top_holder_5_pct,top_holder_6_pct,top_holder_7_pct,top_holder_8_pct,top_holder_9_pct,top_holder_10_pct
0,3K3F9G7zrPd9hPnCDKbBGXwHM85CD1ewLyzNsH51Ry43,TokenkegQfeZyiNwAJbNbGKPFXCWuBvf9Ss623VQ5DA,None,[{'address': 'DyWNa439guZzgca6GFwhHgFjr8iGm9Vq...,None,None,"[{'name': 'Low amount of LP Providers', 'value...",100,None,None,[{'pubkey': '7XQh6FNHiumtgjCXPiDiSnDWoEGd9ZYHr...,False,,None,999999928222742806,9,True,None,Baby Slerf,BABYSLERF,https://ipfs.coinsult.app/ipfs/QmPU63r9RMDM35w...,False,EKJCvZtJh9YMc4TycxfatAUAC4XT6sXcw91wrnLWGegN,,Baby Slerf,BABYSLERF,https://ipfs.coinsult.app/ipfs/QmeZaCHdmPSkqyj...,0,0,11111111111111111111111111111111,60.894393,67.097792,987183.650027,41.6053,9.000001,2.307059,1.647679,1.446064,1.047741,1.0,0.986639,0.966198,0.887712
